In [1]:
from operator import mod
import os 
import sys


sys.path.append('/home/jizhidemifan/Project/optical_film_toolbox')

import pandas as pd 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
import plotly
import plotly.graph_objects as go


from openTSNE import TSNE
from openTSNE.callbacks import ErrorLogger
from common.utils.tsne_utils import plot, MACOSKO_COLORS

from models.fcn_ae import FullyConnectedAutoEncoder
from refractivesqlite import dboperations as DB

In [2]:
save_model = True
show_result = True
show_tSNE = True 

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [47]:
rangeMin = 0.3
rangeMax = 1.0
re = 10

dbpath = "../db_sqlite/refractive.db"
db = DB.Database(dbpath)
matList = db.search_custom('select * from pages')
matPageId = list()
for mat in matList:
  if mat[-3] < rangeMin and mat[-2] > rangeMax:
    print(mat)
    matPageId.append(mat[0])
    
print(matPageId)

30, &#39;main&#39;, &#39;Zn&#39;, &#39;Werner&#39;, &#39;main/Zn/Werner.yml&#39;, 1, 1, 0.017586, 2.479684, 150)
(733, &#39;main&#39;, &#39;Zn&#39;, &#39;Werner-DFT&#39;, &#39;main/Zn/Werner-DFT.yml&#39;, 1, 1, 0.017586, 2.479684, 150)
(735, &#39;main&#39;, &#39;Zr&#39;, &#39;Palm&#39;, &#39;main/Zr/Palm.yml&#39;, 1, 1, 0.226076263, 1.684528687, 685)
(736, &#39;main&#39;, &#39;Zr&#39;, &#39;Querry&#39;, &#39;main/Zr/Querry.yml&#39;, 1, 1, 0.22, 55.5556, 306)
(739, &#39;organic&#39;, &#39;methane&#39;, &#39;Martonchik-liquid-111K&#39;, &#39;organic/CH4 - methane/Martonchik-liquid-111K.yml&#39;, 1, 1, 0.002, 71.43, 43)
(740, &#39;organic&#39;, &#39;methane&#39;, &#39;Martonchik-liquid-90K&#39;, &#39;organic/CH4 - methane/Martonchik-liquid-90K.yml&#39;, 1, 1, 0.002, 71.43, 43)
(741, &#39;organic&#39;, &#39;methane&#39;, &#39;Martonchik-solid-90K&#39;, &#39;organic/CH4 - methane/Martonchik-solid-90K.yml&#39;, 1, 1, 0.002, 100.0, 41)
(742, &#39;organic&#39;, &#39;methane&#39;, &#39;Martonch

In [102]:
input_data = {}

for matId in matPageId:
  mat = db.get_material(matId)
  material_name = mat.pageinfo['book'] + '_' + mat.pageinfo['page']
  material_n = []
  material_k = []

  for i in range(int(rangeMin*1000), int(rangeMax*1000), re):
    try:
        material_n.append(mat.get_refractiveindex(i))
    except:
        material_n.append(0)
    try:
      material_k.append(mat.get_extinctioncoefficient(i))
    except:
      material_k.append(0)

  input_data[material_name] = np.concatenate((np.array(material_n), np.array(material_k)))


num_epochs = 600
batch_size = 256
batch_losses = []


train_dataset = tf.data.Dataset.from_tensor_slices(list(input_data.values())).shuffle(1).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices({'name': list(input_data.keys()),
                                                   'value':list(input_data.values())}).batch(1)

Material main/Ag/Johnson.yml loaded.
Material main/Ag/Yang.yml loaded.
Material main/Ag/Babar.yml loaded.
Material main/Ag/Werner.yml loaded.
Material main/Ag/Hagemann.yml loaded.
Material main/Ag/Ciesielski.yml loaded.
Material main/Ag/Ciesielski-Ge.yml loaded.
Material main/Ag/Ciesielski-Ni.yml loaded.
Material main/Ag/Rakic-BB.yml loaded.
Material main/Ag/Rakic-LD.yml loaded.
Material main/Ag/Werner-DFT.yml loaded.
Material main/Al/Rakic.yml loaded.
Material main/Al/McPeak.yml loaded.
Material main/Al/Hagemann.yml loaded.
Material main/Al/Rakic-BB.yml loaded.
Material main/Al/Rakic-LD.yml loaded.
Material main/AlAs/Rakic.yml loaded.
Material main/GaAs/Adachi.yml loaded.
Material main/GaAs/Rakic.yml loaded.
Material main/GaAs/Ozaki.yml loaded.
Material main/InAs/Adachi.yml loaded.
Material main/Au/Johnson.yml loaded.
Material main/Au/Babar.yml loaded.
Material main/Au/Werner.yml loaded.
Material main/Au/Hagemann.yml loaded.
Material main/Au/Ciesielski.yml loaded.
Material main/Au/Cie

In [116]:
model = FullyConnectedAutoEncoder(latent_dim=20, input_shape=int(((1000-300)/10)*2))

optimizer = tf.keras.optimizers.Adam(1e-4)
global_step = tf.Variable(0)

for epoch in range(num_epochs):
    for step,x in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            x_rec_logits = model(x)
            rec_loss =tf.losses.mean_squared_error(x,x_rec_logits)
            batch_losses.append(rec_loss)
            rec_loss =tf.reduce_mean(rec_loss)
        
        grads=tape.gradient(rec_loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        if step%100==0:
            print(epoch, step, float(rec_loss))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx(&#39;float64&#39;)`. To change just this layer, pass dtype=&#39;float64&#39; to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0 0 7.196256637573242
1 0 6.796089172363281
2 0 6.443813323974609
3 0 6.08891487121582
4 0 5.73411750793457
5 0 5.385976791381836
6 0 5.04655122756958
7 0 4.712028503417969
8 0 4.379741191864014
9 0 4.050856590270996
10 0 3.726564407348633
11 0 3.4064526557922363
12 0 3.0908093452453613
13 0 2.7835521697998047
14 0 2.490626811981201
15 0 2.215254306793213
16 0 1.9567550420761108
17 0 1.7154064178466797
18 0 1.4947631359100342
19 0 1.2974244356155396
20 0 1.1227140426635742
21 0 0.9693574

In [127]:
if show_tSNE:
  tSNE_result = []

  tsne = TSNE(
    perplexity=10,
    metric="euclidean",
    n_jobs=4,
    random_state=42,
    negative_gradient_method="fft",
  )


  for test in test_dataset:
    # print(test)
    name = test['name']
    result = list(model.encode(test['value']))[0]
    tSNE_result.append(result)

  embedding_train = tsne.fit(np.array(tSNE_result))
  # print(embedding_train)
  # plot(embedding_train, list(input_data.keys()))
  # plt.savefig('/home/jizhidemifan/Project/optical_film_toolbox/figure/tSNE.png')

embedding_y = [v[1] for v in embedding_train]
embedding_color = [abs(v[0]) + abs(v[1]) for v in embedding_train]


In [128]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import normalize


ss = QuantileTransformer()
embedding_train_sclar= ss.fit_transform(embedding_train)

embedding_x = [v[0] for v in embedding_train_sclar]
embedding_y = [v[1] for v in embedding_train_sclar]

# embedding_color = [abs(v[0]) + abs(v[1]) for v in embedding_train_sclar]

fig = go.Figure(data=go.Scatter(
                                x=embedding_x,
                                y=embedding_y,
                                text=list(input_data.keys()),
                                marker_color=embedding_color,
                                marker_colorscale='Viridis',
                                marker_size=8,
                                mode='markers'
))

fig.update_layout(
    template="plotly_white",
    autosize=False,
    width=500,
    height=500)

fig.show()
fig.write_html('../figure/tsne/20-hidden-debug.html')